In [ ]:
import alpaca_trade_api as tradeapi
import time
import logging
from datetime import datetime, timezone

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
api_key = "PKW0YNTXIQBYEO0GHQS4"
secret_key = "r0obc1M6nmK1k9aTF7OixLHtGB0PCLBR9RFAJ2Rm"
class PaperTradingBot:
    def __init__(self, api_key, secret_key, base_url='https://paper-api.alpaca.markets'):
        self.api = tradeapi.REST(api_key, secret_key, base_url, api_version='v2')
        self.symbol = 'SPY'  # Example ETF, modify as needed
        self.position_size = 0.5  # 50% of portfolio per position
        self.stop_loss = 0.95  # 5% stop loss
        self.take_profit = 1.05  # 5% take profit

    def market_open(self):
        clock = self.api.get_clock()
        return clock.is_open

    def get_position(self):
        try:
            return self.api.get_position(self.symbol)
        except:
            return None

    def execute_trade(self, signal):
        if not self.market_open():
            logging.warning("Market is closed. No trades executed.")
            return

        current_price = self.api.get_latest_trade(self.symbol).price
        equity = float(self.api.get_account().equity)
        qty = int((equity * self.position_size) / current_price)

        position = self.get_position()

        try:
            if signal == "BUY" and not position:
                order = self.api.submit_order(
                    symbol=self.symbol,
                    qty=qty,
                    side='buy',
                    type='market',
                    time_in_force='gtc',
                    order_class='bracket',
                    stop_loss={'stop_price': current_price * self.stop_loss},
                    take_profit={'limit_price': current_price * self.take_profit}
                )
                logging.info(f"Buy order executed: {order}")

            elif signal == "SELL" and position:
                self.api.close_position(self.symbol)
                logging.info(f"Sell order executed for {position.qty} shares")
  
        except Exception as e:
            logging.error(f"Trade execution failed: {str(e)}")

    def run(self, prediction_model):
        while True:
            try:
                if self.market_open():
                    # Get prediction from your model
                    prediction = prediction_model.predict()
                    
                    # Execute trade based on prediction
                    self.execute_trade(prediction['action'])
                    
                    # Sleep until next market minute
                    next_update = datetime.now(timezone.utc) + timedelta(minutes=1)
                    sleep_time = (next_update - datetime.now(timezone.utc)).total_seconds()
                    time.sleep(sleep_time)
                else:
                    time.sleep(60)  # Check every minute when market is closed
                    
            except Exception as e:
                logging.error(f"Error in main loop: {str(e)}")
                time.sleep(60)

# Usage Example
if __name__ == "__main__":
    # Initialize your prediction model here
    class SampleModel:
        def predict(self):
            # Replace with your model's prediction logic
            return {'action': 'BUY', 'confidence': 0.75}
    
    # Configure with your Alpaca API keys
    bot = PaperTradingBot(
        api_key='YOUR_API_KEY',
        secret_key='YOUR_SECRET_KEY'
    )
    
    bot.run(SampleModel())


In [ ]:
import alpaca_trade_api as tradeapi
import time
import logging
from datetime import datetime, timezone, timedelta

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

class PaperTradingBot:
    def __init__(self, api_key, secret_key, base_url='https://paper-api.alpaca.markets'):
        self.api = tradeapi.REST(api_key, secret_key, base_url, api_version='v2')
        self.stop_loss = 0.95  # 5% stop loss
        self.take_profit = 1.05  # 5% take profit

    def market_open(self):
        clock = self.api.get_clock()
        return clock.is_open

    def get_position(self, symbol):
        try:
            return self.api.get_position(symbol)
        except:
            return None

    def execute_trade(self, signal, symbol, qty):
        if not self.market_open():
            logging.warning("Market is closed. No trades executed.")
            return

        current_price = self.api.get_latest_trade(symbol).price
        position = self.get_position(symbol)

        try:
            if signal == "BUY" and not position:
                order = self.api.submit_order(
                    symbol=symbol,
                    qty=qty,
                    side='buy',
                    type='market',
                    time_in_force='gtc',
                    order_class='bracket',
                    stop_loss={'stop_price': current_price * self.stop_loss},
                    take_profit={'limit_price': current_price * self.take_profit}
                )
                logging.info(f"Buy order executed for {symbol}: {order}")

            elif signal == "SELL" and position:
                self.api.close_position(symbol)
                logging.info(f"Sell order executed for {symbol}: {position.qty} shares")

        except Exception as e:
            logging.error(f"Trade execution failed for {symbol}: {str(e)}")

    def run(self, prediction_model):
        while True:
            try:
                if self.market_open():
                    # Get user input for stock symbol and quantity
                    symbol = input("Enter NSE stock symbol (e.g., RELIANCE.NS): ").upper()
                    qty = int(input("Enter quantity of stocks to trade: "))

                    # Get prediction from your model
                    prediction = prediction_model.predict(symbol)
                    
                    # Execute trade based on prediction
                    self.execute_trade(prediction['action'], symbol, qty)
                    
                    # Sleep until next market minute
                    next_update = datetime.now(timezone.utc) + timedelta(minutes=1)
                    sleep_time = (next_update - datetime.now(timezone.utc)).total_seconds()
                    time.sleep(sleep_time)
                else:
                    logging.info("Market is closed. Waiting for market open.")
                    time.sleep(60)  # Check every minute when market is closed
                    
            except Exception as e:
                logging.error(f"Error in main loop: {str(e)}")
                time.sleep(60)

# Usage Example
if __name__ == "__main__":
    # Initialize your prediction model here
    class SampleModel:
        def predict(self, symbol):
            # Replace with your model's prediction logic
            # This should take the symbol as input and return a prediction
            return {'action': 'BUY', 'confidence': 0.75}
    
    # Configure with your Alpaca API keys
    bot = PaperTradingBot(
        api_key='YOUR_API_KEY',
        secret_key='YOUR_SECRET_KEY'
    )
    
    bot.run(SampleModel())
